In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import subjects
from plotters import violinplot
from subjects import sns_boxplot_kw, stat_kw
from neuropy import plotting
from statannotations.Annotator import Annotator

grpdata = subjects.GroupData()
group = ["NSD", "SD"]

ev_states = grpdata.ev_brainstates
ev_df = grpdata.ev_pooled
ev_bool = ev_df.name.isin(
    [
        "RatKDay1",
        "RatKDay2",
        "RatNDay2",
        "RatSDay2",
        "RatUDay2",
        "RatVDay3",
        "RatNDay1",
        "RatSDay3",
        "RatUDay1",
        "RatVDay2",
        # "RatRDay1",
        "RatRDay2"
    ]
)
ev_df = ev_df[ev_bool]
# ev_df["ev_diff"] = ev_df["ev"] - ev_df["rev"]
ev_pooled = grpdata.ev_in_chunks

fig = plotting.Fig(grid=(16, 8), fontsize=5,constrained_layout=False,axis_lw=0.8,tick_size=2)
filename = subjects.figpath_sd / "figure_supp_expvar"
caption = "Figure 3: MAZE reactivation during POST. \n(A) Explained variance (EV) in POST for each session of SD (orange) and NSD (purple) group with chance levels (gray). Error bars indicate +/- stdev. \n(B) Mean explained variance pooled across SD and NSD sessions. Error bars indicate +/- SEM."



ModuleNotFoundError: No module named 'subjects'

In [41]:
from neuropy.plotting import plot_epochs
from neuropy.core import Epoch
from hfuncs import merge_quiet_wake

errorbar_kw = dict(alpha=0.5, linewidth=0, ec=None)
session_names = ev_df.name.unique()
# animal_names = [_[:4] for _ in session_names]
animal_names = ["RatK","RatN", "RatS", "RatU", "RatV",'RatR']
ev_colors = subjects.colors_sd(1)

subfig = fig.fig.add_subfigure(fig.gs[:7, :4])
axs = subfig.subplots(
    12,
    2,
    sharey="row",
    sharex=True,
    gridspec_kw=dict(height_ratios=[1, 4, 1, 4, 1, 4, 1,4,1,4,1,4]),
)
# axs = axs.T.reshape(-1)


for row, anim in enumerate(animal_names):

    anim_sessions = session_names[[anim in _ for _ in session_names]]
    df_name = ev_df[ev_df.name.isin(anim_sessions)]
    states_name = ev_states[ev_states.name.isin(anim_sessions)]
    sex = df_name.sex.iloc[0]
    grps_name = df_name.grp.unique()[::-1]

    # subfig = fig.fig.add_subfigure(fig.gs[row, :2])
    # axs = subfig.subplots(2, 2, sharey="row", gridspec_kw=dict(height_ratios=[1, 4]))
    # axs = axs.T.reshape(-1)

    for i, sess in enumerate(anim_sessions):
        states = states_name[states_name["name"] == sess]
        df = df_name[df_name["name"] == sess]
        grp = df.grp.unique()[0]

        n_neurons = df["n_neurons"].unique()
        n_pairs = df["n_pairs"].unique()
        t, ev, ev_err, rev, rev_err = (
            df["zt"],
            df["ev"],
            df["ev_std"],
            df["rev"],
            df["rev_std"],
        )

        ax_hypno = axs[2 * row,i]
        ax = axs[2 * row + 1,i]

        # epoch = Epoch(states).duration_slice(min_dur=120/3600)
        epoch = merge_quiet_wake(Epoch(states))
        plot_epochs(
            epoch,
            ax=ax_hypno,
            labels_order=["NREM", "REM", "WK"],
            colors='k',
            collapsed=False,
        )
        ax_hypno.set_xlim([0, 8])
        ax_hypno.set_axis_off()

        ax.fill_between(t, rev - rev_err, rev + rev_err, color="#63c56e", **errorbar_kw)
        ax.plot(t, rev, color="#63c56e")

        if grp == "NSD":
            color = ev_colors[0]
            ax.fill_between(t, ev - ev_err, ev + ev_err, color=color, **errorbar_kw)
            ax.plot(t, ev, color=color, lw=0.8)

        if grp == "SD":
            sd_color = ev_colors[1]
            sd_indx = t <= 5
            ax.fill_between(
                t[sd_indx],
                ev[sd_indx] - ev_err[sd_indx],
                ev[sd_indx] + ev_err[sd_indx],
                color=sd_color,
                **errorbar_kw,
            )
            ax.plot(t[sd_indx], ev[sd_indx], color=sd_color, lw=0.8)

            rs_color = "#5599FF"
            ax.fill_between(
                t[~sd_indx],
                ev[~sd_indx] - ev_err[~sd_indx],
                ev[~sd_indx] + ev_err[~sd_indx],
                color=rs_color,
                **errorbar_kw,
            )
            ax.plot(t[~sd_indx], ev[~sd_indx], color=rs_color, lw=0.8)
            if anim == 'RatR':
                ax.set_ylim([0,0.24])

        ax.text(
            0.4,
            0.8,
            f"{n_neurons[0]} PN,{n_pairs[0]} pairs",
            fontdict=dict(fontsize=5),
            transform=ax.transAxes,
        )
        ax.set_xlim([0, 8])

        if anim == "RatR":
            ax.set_ylim(0, 0.25)

        sex_symbol = "$\u2640$" if sex == "female" else "$\u2642$"
        # sex_color = "#F50057" if sex == "female" else "#2962FF"
        if i == 0:
            ax.set_ylabel(f"EV\n({anim} {sex_symbol})")
            # fig.legend(ax, [name + sex_symbol], color='k', x=-0.4, y=1)

        # ax.set_xticks([1, 3, 5, 7, 9])



In [42]:
sessions = subjects.pf_sess()

subfig = fig.fig.add_subfigure(fig.gs[:6, 4:6])
axs = subfig.subplots(8,2)
axs = axs.reshape(-1)
# axs = axs.T.reshape(-1)



for s, sess in enumerate(sessions):

    maze= sess.paradigm['maze'].flatten()
    pos = sess.position.time_slice(*maze)
    x= pos.x
    y= pos.y

    ax=axs[s]
    ax.plot(x,y,'gray',lw=0.5)
    ax.set_title(sess.name)
    ax.axis('off')

    

#Sessions = 13


In [43]:
sessions

[ProcessData(RatK_Day2_2019-08-08_04-00-00.xml),
 ProcessData(RatN_Day2_2019-10-11_03-58-54.xml),
 ProcessData(RatS-Day2NSD-2020-11-27_10-22-29.xml),
 ProcessData(RatU_Day2NSD_2021-07-24_08-16-38.xml),
 ProcessData(RatV_Day1NSD_2021-10-02_08-10-23.xml),
 ProcessData(RatV_Day3NSD_2021-10-07_08-10-12.xml),
 ProcessData(RatK_Day1_2019-08-06_03-44-01.xml),
 ProcessData(RatN_Day1_2019-10-09_03-52-32.xml),
 ProcessData(RatS_Day3SD_2020-11-29_07-53-30.xml),
 ProcessData(RatU_Day1SD_2021-07-22_07-55-46.xml),
 ProcessData(RatU_Day4SD_2021-07-29_08-23-06.xml),
 ProcessData(RatV_Day2SD_2021-10-04_08-02-10.xml),
 ProcessData(RatR_Day2SD_2021-05-23_09-07-18.xml)]

In [44]:
fig.savefig(filename)

meta NOT subset; don't know how to subset; dropped
